# step 1: Loading the data

In [3]:
with open("../data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()] # strip the whitespaces
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [8]:
# creating the tokens from the raw text

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

In [9]:
print(preprocessed[:20])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was']


# step 2: convert tokens into token IDs

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

vocab_size

1130

In [11]:
vocab = { tokens: integers for  integers, tokens in enumerate(all_words)}

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


# Step 3: Creating a Tokenizer_Class

In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab) -> None:
        self.str_to_int = vocab
        self.int_to_str = { i: s  for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        
        ids = [self.str_to_int[item] for item in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
        

In [15]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [16]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


### what if we encounter a word not in the vocabulary?

In [17]:
# text = "Hello, do you like tea?"
# print(tokenizer.encode(text))

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())


1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab) -> None:
        self.str_to_int = vocab
        self.int_to_str = { i: s  for s, i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        
        preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
        
        ids = [self.str_to_int[item] for item in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
        

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
ids= tokenizer.encode(text)
ids

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
print(tokenizer.decode(ids))


<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


# step 4: Byte Pair Encoding (BPE) 

In [25]:
!pip install tiktoken

In [26]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.11.0


In [27]:
tokenizer = tiktoken.get_encoding('gpt2')

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)


integers = tokenizer.encode(text, allowed_special={'<|endoftext|>'})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
print(tokenizer.decode(integers))

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [30]:
integers = tokenizer.encode("jhusk ier")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[73, 7537, 74, 220, 959]
jhusk ier


# Step 5:  Creating Input-Target Pairs

In [31]:
with open("../data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [32]:
enc_sample = enc_text[50:]


In [33]:
context_size = 4 #length of the input
#The context_size of 4 means that the model is trained to look at a sequence of 4 words (or tokens) 
#to predict the next word in the sequence. 
#The input x is the first 4 tokens [1, 2, 3, 4], and the target y is the next 4 tokens [2, 3, 4, 5]

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [35]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))
    

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


# Step 6: Implementing Dataset and Dataloader

<div class="alert alert-block alert-info">
    
Step 1: Tokenize the entire text
    
Step 2: Use a sliding window to chunk the book into overlapping sequences of max_length

Step 3: Return the total number of rows in the dataset

Step 4: Return a single row from the dataset
</div>

In [96]:
from torch.utils.data import Dataset, DataLoader
import torch

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride) -> None: # stride = sliding window
        self.input_ids = []
        self.target_ids = []
        
        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunks = token_ids[i:i+max_length]
            target_chunks = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunks))
            self.target_ids.append(torch.tensor(target_chunks))
        
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
        
        
        

<div class="alert alert-block alert-info">
    
Step 1: Initialize the tokenizer

Step 2: Create dataset

Step 3: drop_last=True drops the last batch if it is shorter than the specified batch_size to prevent loss spikes
during training

Step 4: The number of CPU processes to use for preprocessing
    
</div>

In [97]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    tokenizer = tiktoken.get_encoding('gpt2')
    
    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader


In [98]:
with open("../data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [99]:
import torch
print("PyTorch version:", torch.__version__)
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.8.0+cpu
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [100]:
input, output= next(data_iter)
print(input)
print(output)

tensor([[ 367, 2885, 1464, 1807]])
tensor([[2885, 1464, 1807, 3619]])


In [101]:
dataloader = create_dataloader_v1(raw_text, batch_size=4, max_length=8, stride=4, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [10899,  2138,   257,  7026, 15632,   438,  2016,   257],
        [15632,   438,  2016,   257,   922,  5891,  1576,   438]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 3619,   402,   271, 10899,  2138,   257,  7026, 15632],
        [ 2138,   257,  7026, 15632,   438,  2016,   257,   922],
        [  438,  2016,   257,   922,  5891,  1576,   438,   568]])]


# Step 7: Creating Tokens Embeddings

In [102]:
input_ids = torch.tensor([2, 3, 5, 1])


In [103]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [104]:
print(embedding_layer.weight)


Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [105]:
print(embedding_layer(torch.tensor([3])))


tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [106]:
# extract layer of input

print(embedding_layer(input_ids))


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


# Step 8: Creating Positional Embeddings

In [107]:
# gpt 2 model 
vocab_size = 50257
output_dim = 256 # actual dimension is start from 768

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [108]:
token_embedding_layer(torch.tensor([1]))

tensor([[ 1.2277, -0.4297, -2.2121, -0.3780,  0.9838, -1.0895, -0.6378, -0.6031,
         -1.7753, -0.7490,  0.2781, -0.9621, -0.4223, -1.1036,  0.8398, -1.0029,
         -0.2835, -0.3767, -0.0306, -0.0894, -0.1965, -0.9713,  0.2790, -0.7587,
          1.0669, -0.2985,  0.8558,  1.6098, -1.1893,  1.1677,  0.6220,  2.5737,
         -1.6179,  0.2265, -0.4382,  0.3265, -1.5786, -1.3995,  0.2425,  0.3648,
         -1.1753,  1.7825,  1.7524, -0.2135,  0.4095,  0.0465,  0.5468,  1.1478,
         -0.8614,  0.5338,  0.9376, -0.9225,  0.7047, -0.2722,  0.0781, -0.1134,
          2.3902, -1.4256, -0.4619, -1.5539, -0.3338,  0.2405, -0.0334,  1.5544,
         -0.2936, -1.8027, -0.6933,  1.7409,  0.2698,  0.9595,  0.7744,  1.8721,
          1.0264, -0.5670, -0.2658, -1.1116, -1.3696, -0.6534, -2.1587,  0.8093,
          1.8388, -0.9473,  0.1419,  0.3696, -0.0174, -0.9575,  1.2968,  0.6833,
          0.4343, -0.1340, -2.1467, -1.7984, -0.6822, -0.5191,  0.1669, -1.2620,
         -0.2443,  0.1327,  

In [121]:
# data loader 
max_length = 4

dataloader = create_dataloader_v1(
    raw_text, batch_size=8 ,   max_length= max_length, 
    stride= max_length, shuffle= False
)

data_iter = iter(dataloader)

In [122]:
ip_first, op_first = next(data_iter)

In [123]:
print("Token IDs:\n", op_first)
print("\nInputs shape:\n", op_first.shape)

Token IDs:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])

Inputs shape:
 torch.Size([8, 4])


In [124]:
print(ip_first.shape)

torch.Size([8, 4])


In [127]:
token_embeddings = token_embedding_layer(ip_first)

print(token_embeddings.shape)



torch.Size([8, 4, 256])


In [128]:
# creating positional embedding layer
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [ ]:
pos_embedding_vectors = pos_embedding_layer(torch.arange(context_length))

pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [130]:
pos_embedding_vectors, pos_embedding_vectors.shape

(tensor([[-0.7261, -0.0688, -0.7009,  ..., -0.0921, -0.1557,  2.3573],
         [-0.8398,  0.2168,  1.0688,  ...,  0.3542, -1.1738,  1.0604],
         [ 0.9218, -0.1852, -1.0204,  ...,  0.0693, -0.5336,  0.7380],
         [-1.0855,  1.0690, -1.4043,  ..., -0.6755, -1.2119,  2.2913]],
        grad_fn=<EmbeddingBackward0>),
 torch.Size([4, 256]))

In [131]:
input_embeddings = token_embeddings + pos_embeddings

In [132]:
pos_embedding_vectors.unsqueeze(0).shape

torch.Size([1, 4, 256])

# 